In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
import numpy as np

In [2]:
faults = pd.read_csv("../data/J1939Faults.csv", low_memory = False)
diagnostics = pd.read_csv("../data/VehicleDiagnosticOnboardData.csv")
sfc = pd.read_excel("../data/Service Fault Codes_1_0_0_167.xlsx")

C:\Users\cavin\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [3]:
pd.set_option('display.max_columns', None)

In [4]:
faults_prepped = faults.dropna(axis=1, how='all').dropna(axis=0, how='all')

In [7]:
exclude_coords = [
    (36.0666667, -86.4347222),
    (35.5883333, -86.4438888),
    (36.1950, -83.174722)
]
def haversine(lat1, lon1, lat2, lon2):
    R = 3958.7  # Radius of Earth in km - roughly 3958.7 miles
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2 # this is the haversine formula that's accounting for the curvature of the earth. think calculus & a polar coordinate system.
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)) # we're calculating the central angle between the two points
    return R * c # calculates the arc length (distance) we're multiplying the radius of earth and the central angle
threshold = 0.5  # Distance in km - this is roughly about 3.1 miles
distances = np.array([
    haversine(faults_prepped['Latitude'], faults_prepped['Longitude'], lat, lon)
    for lat, lon in exclude_coords
]) # I'm calculating the difference between the excluded coordinates and geo coordinate in my dataframe. the array is contains the distances as scalar quantities (floats).
# which rows are outside all vicinities
outside_vicinity = (distances > threshold).all(axis=0) # boolean array true if its outside vicinity
faults_prepped = faults_prepped[outside_vicinity]

In [8]:
diagnostics_pivot = diagnostics.pivot(index=['FaultId'], columns='Name', values='Value')

In [9]:
data = diagnostics_pivot.merge(faults_prepped, left_on = 'FaultId', right_on = 'RecordID', how = 'inner')

In [10]:
columns_to_change = ['AcceleratorPedal', 
                     'BarometricPressure',
                     'CruiseControlSetSpeed',
                     'DistanceLtd',
                     'EngineCoolantTemperature',
                     'EngineLoad',
                     'EngineOilPressure',
                     'EngineOilTemperature',
                     'EngineRpm',
                     'EngineTimeLtd',
                     'FuelLevel',
                     'FuelLtd',
                     'FuelRate',
                     'FuelTemperature',
                     'IntakeManifoldTemperature',
                     'Speed',
                     'SwitchedBatteryVoltage',
                     'TurboBoostPressure',
                    ]

In [11]:
for column in columns_to_change:
    data[column] = data[column].str.replace(',', '.').astype(np.float64)

In [12]:
data['LocationTimeStamp'] = pd.to_datetime(data['LocationTimeStamp'])
data['EventTimeStamp'] = pd.to_datetime(data['EventTimeStamp'])

In [13]:
data = data.drop(['ecuModel','ecuMake', 'MCTNumber', 'ServiceDistance'], axis=1)

In [22]:
data['time_derate'] = data.loc[data['spn'] == 5246]['EventTimeStamp']
data = data.sort_values('EventTimeStamp')

data['time_derate'] = data.groupby('EquipmentID')['time_derate'].bfill()
data['time_until_derate'] = data['time_derate'] - data['EventTimeStamp']

data['target'] = (data['time_until_derate'] > '2 hour') & (data['time_until_derate'] < '24 hour')

In [23]:
data.to_csv('../data/data_clean.csv')